# Pre Processing the Datasets

There are 3 steps:
1. Processing all Scripts
2. Merging Ratings and Viewerships
3. Saving final CSV files

## 1. Processing the Scripts

In [1]:
import re
import glob
import pandas as pd

In [2]:
def process_scripts(path):
    df = []

    def read_script(pth):
        S_E = pth.split()[0][13:]
        season = int(S_E[1:3])
        episode = int(S_E[4:6])
        title = pth.split(" ", maxsplit=1)[1][:-4]

        foo = []

        with open(pth, 'r') as fh:
            lines = fh.readlines()

        for line in lines:
            temp = re.sub("[\(\[].*?[\)\]]", "", line)
            # Ignore lines that start with '[', because they set up locaiton
            if not temp:
                continue

            if temp[0] == '[' or temp[0] == '(' or temp[0] == '\n' or temp[0] == '{':
                continue
            # Ignore lines without a ':', because no speaker
            if ':' not in temp:
                continue
            speaker, dialog = temp.split(':', maxsplit=1)
            speaker = speaker.strip()
            dialog = dialog.strip('\n')
            foo.append([season, episode, title, speaker, dialog])

        return foo
    
    scripts = glob.glob("data/scripts/*.txt")
    
    for script in scripts:
        print("Processing ",script)
        data = read_script(script)
        for foo in data:
            df.append(foo)

    df = pd.DataFrame(df, columns=["Season", "Episode", "Title", "Speaker", "Dialog"])

    return df

In [3]:
df = process_scripts("data/scripts")

Processing  data/scripts/S04E10 The Girl From Poughkeepsie.txt
Processing  data/scripts/S07E06 The Nap Partners.txt
Processing  data/scripts/S06E03 Rosss Denial.txt
Processing  data/scripts/S03E17 The Ski Trip.txt
Processing  data/scripts/S05E17 Rachels Inadvertant Kiss.txt
Processing  data/scripts/S01E18 All The Poker.txt
Processing  data/scripts/S06E17 Unagi.txt
Processing  data/scripts/S02E18 Dr. Ramoray Dies.txt
Processing  data/scripts/S07E19 Ross And Monicas Cousin.txt
Processing  data/scripts/S05E22 Joeys Big Break.txt
Processing  data/scripts/S09E19 Rachels Dream.txt
Processing  data/scripts/S08E22 Rachel Is Late.txt
Processing  data/scripts/S04E16 The Fake Party.txt
Processing  data/scripts/S08E16 Joey Tells Rachel.txt
Processing  data/scripts/S02E19 Eddie Wont Go.txt
Processing  data/scripts/S05E07 Ross Moves In.txt
Processing  data/scripts/S09E20 The Soap Opera Party.txt
Processing  data/scripts/S02E01 Rosss New Girlfriend.txt
Processing  data/scripts/S05E06 The Yeti.txt
Pro

Processing  data/scripts/S06E11 The Apothecary Table.txt
Processing  data/scripts/S05E05 The Kips.txt
Processing  data/scripts/S04E09 Theyre Going To Party.txt
Processing  data/scripts/S09E18 The Lottery.txt
Processing  data/scripts/S09E04 The Sharks.txt
Processing  data/scripts/S05E15 The Girl Who Hits Joey.txt
Processing  data/scripts/S01E02 The Sonogram At The End.txt
Processing  data/scripts/S06E12 The Joke.txt
Processing  data/scripts/S01E14 The Candy Hearts.txt
Processing  data/scripts/S03E10 Rachel Quits.txt
Processing  data/scripts/S05E18 Rachel Smokes.txt
Processing  data/scripts/S03E12 All The Jealousy.txt
Processing  data/scripts/S01E23 The Birth.txt
Processing  data/scripts/S08E10 Monicas Boots.txt
Processing  data/scripts/S02E05 Five Steaks And An Eggplant.txt
Processing  data/scripts/S04E18 Rachels New Dress.txt
Processing  data/scripts/S02E20 Old Yeller Dies.txt
Processing  data/scripts/S10E11 The Stripper Cries.txt
Processing  data/scripts/S07E04 Rachels Assistant.txt
P

In [4]:
print(df.shape)
df.head()

(61603, 5)


,Season,Episode,Title,Speaker,Dialog
0,4,10,The Girl From Poughkeepsie,Written by,Scott Silveri
1,4,10,The Girl From Poughkeepsie,Ross,Hey!
2,4,10,The Girl From Poughkeepsie,"Chandler, Joey, and Phoebe",Hey!
3,4,10,The Girl From Poughkeepsie,Ross,"I’m sorry I’m late, did I miss anything?"
4,4,10,The Girl From Poughkeepsie,Phoebe,Joey stuffing 15 Oreos in his mouth.


In [5]:
print("Number of speakers: ", len(df.Speaker.unique()))
df.Speaker.unique()

Number of speakers:  949


array(['Written by', 'Ross', 'Chandler, Joey, and Phoebe', 'Phoebe',
       'Chandler', 'Joey', 'Rachel', 'All', 'Monica', 'Drew', 'Mike',
       'Monica, Rachel, and Joey', 'A Waiter', 'The Waiter',
       'The Conductor', 'The Woman From Poughkeepsie', 'Kitchen Worker',
       'Woman On Train', 'Woman', 'Joey and Ross', 'Ross and Joey',
       'Julie', 'The Potential Roommate', 'Gunther', 'Monica and Phoebe',
       'The Girls', 'Carol', 'Chandler and Joey',
       'Chandler, Phoebe, and Joey', "Gary's Radio", 'Gary',
       'The Old Man', 'The Little Girl', 'Mr. Zelner', 'The Man',
       'The Hot Girl', 'Jen', 'Chandler and Ross', 'Girls', 'Guys',
       'Phoebe and Rachel', 'Aunt Iris', 'Teleplay by', 'Story by',
       'Receptionist', "Joey's Look-A-Like", 'Carl', 'The Doctor',
       'Janice', 'The Instructor', 'Janice’s Voice', 'AMBER',
       'DR. REMORE', 'ROSS', 'MONICA', 'RACHEL', 'JOEY', 'CHANDLER',
       'PHOEBE', 'EDDIE', 'RICHARD', 'TILLY', 'WRITER', 'DELIVERY GUY',
  

In [6]:
# Remove all speakers with less than 'n' lines of dialogue
def analyse_speakers_to_remove(df, n):
    speakers_to_remove = []
    lines_of_speaker = []
    for speaker in df.Speaker.unique():
        lines = len(df[df.Speaker == speaker])
        if lines < n:
            speakers_to_remove.append(speaker)
            lines_of_speaker.append(lines)
    return speakers_to_remove, lines_of_speaker


for n in [10,25,50,75,100,125]:
    print('-'*50)
    print(f"For {n} lines: ")
    speakers_to_remove, lines_of_speaker = analyse_speakers_to_remove(df, n)
    print(f"Remove {len(speakers_to_remove)} speakers and {sum(lines_of_speaker)} lines of dialogue")
    print(f"i.e., removing {(sum(lines_of_speaker) / df.shape[0])*100}% of all lines from corpus.")
    print(f"i.e., removing {(len(speakers_to_remove) / len(df.Speaker.unique()))*100}% of all speakers")
print('-'*50)

--------------------------------------------------
For 10 lines: 
Remove 695 speakers and 1964 lines of dialogue
i.e., removing 3.188156420953525% of all lines from corpus.
i.e., removing 73.23498419388831% of all speakers
--------------------------------------------------
For 25 lines: 
Remove 837 speakers and 4129 lines of dialogue
i.e., removing 6.702595652809117% of all lines from corpus.
i.e., removing 88.19810326659642% of all speakers
--------------------------------------------------
For 50 lines: 
Remove 888 speakers and 5826 lines of dialogue
i.e., removing 9.457331623459897% of all lines from corpus.
i.e., removing 93.57218124341412% of all speakers
--------------------------------------------------
For 75 lines: 
Remove 906 speakers and 6908 lines of dialogue
i.e., removing 11.213739590604353% of all lines from corpus.
i.e., removing 95.46891464699684% of all speakers
--------------------------------------------------
For 100 lines: 
Remove 915 speakers and 7712 lines of di

In [7]:
# Removing people with < 50 lines of dialogue seems like a good balance
# Getting list of people with > 50 lines of dialogue

speakers_to_remove, _ = analyse_speakers_to_remove(df, 50)
s = set(speakers_to_remove)
final_list = [[x, df[df.Speaker == x].shape[0]] for x in df.Speaker.unique() if x not in s]
final_list.sort(key=lambda x: x[1])

In [8]:
print(len(final_list))
final_list[::-1]

61


[['Rachel', 8510],
 ['Ross', 8251],
 ['Chandler', 7687],
 ['Monica', 7650],
 ['Joey', 7561],
 ['Phoebe', 6830],
 ['ROSS', 836],
 ['CHANDLER', 693],
 ['MONICA', 659],
 ['JOEY', 643],
 ['RACHEL', 638],
 ['PHOEBE', 562],
 ['Mike', 327],
 ['All', 299],
 ['Janice', 209],
 ['Charlie', 190],
 ['Emily', 167],
 ['Mr. Geller', 164],
 ['Carol', 156],
 ['Written by', 152],
 ['Tag', 146],
 ['RACH', 140],
 ['Paul', 133],
 ['RICHARD', 128],
 ['Richard', 126],
 ['Frank', 126],
 ['Mrs. Geller', 126],
 ['David', 120],
 ['Gunther', 120],
 ['Amy', 116],
 ['Mona', 111],
 ['MNCA', 107],
 ['Woman', 105],
 ['Pete', 103],
 ['Joshua', 98],
 ['Gary', 96],
 ['Elizabeth', 94],
 ['Janine', 93],
 ['Susan', 90],
 ['Kathy', 88],
 ['Jill', 83],
 ['CHAN', 82],
 ['PHOE', 80],
 ['Joanna', 73],
 ['Ursula', 71],
 ['Ben', 68],
 ['Eric', 68],
 ['Erica', 67],
 ['Gavin', 64],
 ['EDDIE', 64],
 ['Dr. Green', 63],
 ['Kate', 61],
 ['Danny', 58],
 ['Director', 57],
 ['Man', 56],
 ['Guy', 54],
 ['Steve', 53],
 ['Nurse', 53],
 ['Mark'

In [9]:
# Since character's have multiple spellings, standardize them

alt_spellings = {
    'Rachel': ['RACHEL', 'RACH'],
    'Ross': ['ROSS'],
    'Chandler': ['CHANDLER', 'CHAN'],
    'Monica': ['MONICA', 'MNCA'],
    'Joey': ['JOEY'],
    'Phoebe': ['PHOEBE', 'PHOE'],
    'Richard': ['RICHARD']
}


for speaker, alts in alt_spellings.items():
    for alt in alts:
        df.Speaker.mask(df.Speaker == alt, speaker, inplace=True)
        
# Check new stats for people with > 50 lines
speakers_to_remove, _ = analyse_speakers_to_remove(df, 50)
s = set(speakers_to_remove)
final_list = [[x, df[df.Speaker == x].shape[0]] for x in df.Speaker.unique() if x not in s]
final_list.sort(key=lambda x: x[1])
print(len(final_list))
final_list[::-1]

50


[['Rachel', 9288],
 ['Ross', 9087],
 ['Chandler', 8462],
 ['Monica', 8416],
 ['Joey', 8204],
 ['Phoebe', 7472],
 ['Mike', 327],
 ['All', 299],
 ['Richard', 254],
 ['Janice', 209],
 ['Charlie', 190],
 ['Emily', 167],
 ['Mr. Geller', 164],
 ['Carol', 156],
 ['Written by', 152],
 ['Tag', 146],
 ['Paul', 133],
 ['Frank', 126],
 ['Mrs. Geller', 126],
 ['David', 120],
 ['Gunther', 120],
 ['Amy', 116],
 ['Mona', 111],
 ['Woman', 105],
 ['Pete', 103],
 ['Joshua', 98],
 ['Gary', 96],
 ['Elizabeth', 94],
 ['Janine', 93],
 ['Susan', 90],
 ['Kathy', 88],
 ['Jill', 83],
 ['Joanna', 73],
 ['Ursula', 71],
 ['Ben', 68],
 ['Eric', 68],
 ['Erica', 67],
 ['Gavin', 64],
 ['EDDIE', 64],
 ['Dr. Green', 63],
 ['Kate', 61],
 ['Danny', 58],
 ['Director', 57],
 ['Man', 56],
 ['Guy', 54],
 ['Steve', 53],
 ['Nurse', 53],
 ['Mark', 52],
 ['Doug', 50],
 ['Will', 50]]

In [10]:
speakers_to_remove

['Chandler, Joey, and Phoebe',
 'Drew',
 'Monica, Rachel, and Joey',
 'A Waiter',
 'The Waiter',
 'The Conductor',
 'The Woman From Poughkeepsie',
 'Kitchen Worker',
 'Woman On Train',
 'Joey and Ross',
 'Ross and Joey',
 'Julie',
 'The Potential Roommate',
 'Monica and Phoebe',
 'The Girls',
 'Chandler and Joey',
 'Chandler, Phoebe, and Joey',
 "Gary's Radio",
 'The Old Man',
 'The Little Girl',
 'Mr. Zelner',
 'The Man',
 'The Hot Girl',
 'Jen',
 'Chandler and Ross',
 'Girls',
 'Guys',
 'Phoebe and Rachel',
 'Aunt Iris',
 'Teleplay by',
 'Story by',
 'Receptionist',
 "Joey's Look-A-Like",
 'Carl',
 'The Doctor',
 'The Instructor',
 'Janice’s Voice',
 'AMBER',
 'DR. REMORE',
 'TILLY',
 'WRITER',
 'DELIVERY GUY',
 'RACHEL and MONICA',
 'DR HORTON',
 'INTERCOM',
 'DR. HORTON',
 'AMGER',
 'Estelle',
 'Cassie',
 'The Casting Director',
 'The Other Woman',
 'The Director',
 'Dr. Miller',
 'Stanley',
 'The Grip',
 'The Husband',
 'Story',
 'Directed by',
 'Actress/Olivia',
 'Joey/Drake',
 '

In [11]:
# Delete the 'Director' and 'Writen by' rows, along with all other rows in 'speakers_to_remove'
speakers_to_remove.append('Director')
speakers_to_remove.append('Written by')

for speaker in speakers_to_remove:
    df.drop(df[df.Speaker == speaker].index, inplace=True)
df.Speaker.unique()

array(['Ross', 'Phoebe', 'Chandler', 'Joey', 'Rachel', 'All', 'Monica',
       'Mike', 'Woman', 'Gunther', 'Carol', 'Gary', 'Janice', 'EDDIE',
       'Richard', 'Emily', 'Joshua', 'Danny', 'Charlie', 'Eric', 'Will',
       'Nurse', 'Man', 'Mrs. Geller', 'Steve', 'Guy', 'Dr. Green',
       'Erica', 'David', 'Mr. Geller', 'Elizabeth', 'Jill', 'Frank',
       'Mona', 'Mark', 'Tag', 'Doug', 'Ben', 'Janine', 'Pete', 'Ursula',
       'Kathy', 'Paul', 'Susan', 'Kate', 'Gavin', 'Joanna', 'Amy'],
      dtype=object)

In [12]:
df.head()

,Season,Episode,Title,Speaker,Dialog
1,4,10,The Girl From Poughkeepsie,Ross,Hey!
3,4,10,The Girl From Poughkeepsie,Ross,"I’m sorry I’m late, did I miss anything?"
4,4,10,The Girl From Poughkeepsie,Phoebe,Joey stuffing 15 Oreos in his mouth.
5,4,10,The Girl From Poughkeepsie,Ross,15? Your personal best!
6,4,10,The Girl From Poughkeepsie,Phoebe,Where were you?


In [13]:
# Remove Punctuation and make everything lower case
df.Dialog = df.Dialog.apply(lambda s: ' '.join(re.sub("[.,!?:;-=...\"@#_]", " ", s).split()).lower())
df.Dialog = df.Dialog.apply(lambda s: ' '.join(re.sub("['’‘—…]", "", s).split()))
df

,Season,Episode,Title,Speaker,Dialog
1,4,10,The Girl From Poughkeepsie,Ross,hey
3,4,10,The Girl From Poughkeepsie,Ross,im sorry im late did i miss anything
4,4,10,The Girl From Poughkeepsie,Phoebe,joey stuffing 15 oreos in his mouth
5,4,10,The Girl From Poughkeepsie,Ross,15 your personal best
6,4,10,The Girl From Poughkeepsie,Phoebe,where were you
...,...,...,...,...,...
61598,3,25,At The Beach,Ross,hi
61599,3,25,At The Beach,Chandler,hi there
61600,3,25,At The Beach,Monica,thats that weird voice again
61601,3,25,At The Beach,Chandler,okay okay let me try it again youre gonna wann...


In [14]:
# Creating S-E column for unique episode identification down the line
df.Season = df.Season.apply(str)
df.Season = df.Season.str.zfill(2)
df.Episode = df.Episode.apply(str)
df.Episode = df.Episode.str.zfill(2)
df['S-E'] = df.Season + '-' + df.Episode
df

,Season,Episode,Title,Speaker,Dialog,S-E
1,04,10,The Girl From Poughkeepsie,Ross,hey,04-10
3,04,10,The Girl From Poughkeepsie,Ross,im sorry im late did i miss anything,04-10
4,04,10,The Girl From Poughkeepsie,Phoebe,joey stuffing 15 oreos in his mouth,04-10
5,04,10,The Girl From Poughkeepsie,Ross,15 your personal best,04-10
6,04,10,The Girl From Poughkeepsie,Phoebe,where were you,04-10
...,...,...,...,...,...,...
61598,03,25,At The Beach,Ross,hi,03-25
61599,03,25,At The Beach,Chandler,hi there,03-25
61600,03,25,At The Beach,Monica,thats that weird voice again,03-25
61601,03,25,At The Beach,Chandler,okay okay let me try it again youre gonna wann...,03-25


In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55568 entries, 1 to 61602
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Season   55568 non-null  object
 1   Episode  55568 non-null  object
 2   Title    55568 non-null  object
 3   Speaker  55568 non-null  object
 4   Dialog   55568 non-null  object
 5   S-E      55568 non-null  object
dtypes: object(6)
memory usage: 3.0+ MB


## 2. Merging Ratings and Viewership

In [16]:
ratings = pd.read_csv('data/ratings.csv')
print(ratings.shape)
ratings.head()

(236, 9)


,Year_of_prod,Season,Episode,Episode_Title,Duration,Summary,Director,Stars,Votes
0,1994,1,1,The One Where Monica Gets a Roommate: The Pilot,22,"Monica and the gang introduce Rachel to the ""r...",James Burrows,8.3,7440
1,1994,1,2,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,James Burrows,8.1,4888
2,1994,1,3,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,James Burrows,8.2,4605
3,1994,1,4,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,James Burrows,8.1,4468
4,1994,1,5,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Pamela Fryman,8.5,4438


In [17]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Year_of_prod   236 non-null    int64  
 1   Season         236 non-null    int64  
 2   Episode        236 non-null    int64  
 3   Episode_Title  236 non-null    object 
 4   Duration       236 non-null    int64  
 5   Summary        236 non-null    object 
 6   Director       236 non-null    object 
 7   Stars          236 non-null    float64
 8   Votes          236 non-null    int64  
dtypes: float64(1), int64(5), object(3)
memory usage: 16.7+ KB


In [18]:
ratings.Season = ratings.Season.apply(str)
ratings.Episode = ratings.Episode.apply(str)

ratings.Season = ratings.Season.str.zfill(2)
ratings.Episode = ratings.Episode.str.zfill(2)

ratings['S-E'] = ratings.Season + '-' + ratings.Episode
ratings.head()

,Year_of_prod,Season,Episode,Episode_Title,Duration,Summary,Director,Stars,Votes,S-E
0,1994,01,01,The One Where Monica Gets a Roommate: The Pilot,22,"Monica and the gang introduce Rachel to the ""r...",James Burrows,8.3,7440,01-01
1,1994,01,02,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,James Burrows,8.1,4888,01-02
2,1994,01,03,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,James Burrows,8.2,4605,01-03
3,1994,01,04,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,James Burrows,8.1,4468,01-04
4,1994,01,05,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Pamela Fryman,8.5,4438,01-05


In [19]:
print(len(ratings['S-E'].unique()))
ratings['S-E'].unique()

236


array(['01-01', '01-02', '01-03', '01-04', '01-05', '01-06', '01-07',
       '01-08', '01-09', '01-10', '01-11', '01-12', '01-13', '01-14',
       '01-15', '01-16', '01-17', '01-18', '01-19', '01-20', '01-21',
       '01-22', '01-23', '01-24', '02-01', '02-02', '02-03', '02-04',
       '02-05', '02-06', '02-07', '02-08', '02-09', '02-10', '02-11',
       '02-12', '02-13', '02-14', '02-15', '02-16', '02-17', '02-18',
       '02-19', '02-20', '02-21', '02-22', '02-23', '02-24', '03-01',
       '03-02', '03-03', '03-04', '03-05', '03-06', '03-07', '03-08',
       '03-09', '03-10', '03-11', '03-12', '03-13', '03-14', '03-15',
       '03-16', '03-17', '03-18', '03-19', '03-20', '03-21', '03-22',
       '03-23', '03-24', '03-25', '04-01', '04-02', '04-03', '04-04',
       '04-05', '04-06', '04-07', '04-08', '04-09', '04-10', '04-11',
       '04-12', '04-13', '04-14', '04-15', '04-16', '04-17', '04-18',
       '04-19', '04-20', '04-21', '04-22', '04-23', '04-24', '05-01',
       '05-02', '05-

In [20]:
viewership = pd.read_csv('data/viewership.csv')
print(viewership.shape)
viewership.head()

(229, 10)


,Date,Episode,Title,Directed by,Written by,Duration,Summary,Rating/Share,U.S. viewers,Prod.\ncode
0,09/22/94,01-01,The Pilot,James Burrows,David Crane & Marta Kauffman,22.0,"Monica and the gang introduce Rachel to the ""r...",14.7/23,21.5 million,456650
1,09/29/94,01-02,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,22.0,Ross finds out his ex-wife is pregnant. Rachel...,14.0/22,20.2 million,456652
2,10/06/94,01-03,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,22.0,Monica becomes irritated when everyone likes h...,13.6/22,19.5 million,456651
3,10/13/94,01-04,The One with George Stephanopoulos,James Burrows,Alexa Junge,22.0,Joey and Chandler take Ross to a hockey game t...,13.7/22,19.7 million,456654
4,10/20/94,01-05,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,22.0,"Eager to spend time with Rachel, Ross pretends...",12.9/20,18.6 million,456653


In [21]:
print(len(viewership['Episode'].unique()))
viewership['Episode'].unique()

228


array(['01-01', '01-02', '01-03', '01-04', '01-05', '01-06', '01-07',
       '01-08', '01-09', '01-10', '01-11', '01-12', '01-13', '01-14',
       '01-15', '01-16', '01-17', '01-18', '01-19', '01-20', '01-21',
       '01-22', '01-23', '01-24', '02-01', '02-02', '02-03', '02-04',
       '02-05', '02-06', '02-07', '02-08', '02-09', '02-10', '02-11',
       '02-12\n02-13', '02-14', '02-15', '02-16', '02-17', '02-18',
       '02-19', '02-20', '02-21', '02-22', '02-23', '02-24', '03-01',
       '03-02', '03-03', '03-04', '03-05', '03-06', '03-07', '03-08',
       '03-09', '03-10', '03-11', '03-12', '03-13', '03-14', '03-15',
       '03-16', '03-17', '03-18', '03-19', '03-20', '03-21', '03-22',
       '03-23', '03-24', '03-25', '04-01', '04-02', '04-03', '04-04',
       '04-05', '04-06', '04-07', '04-08', '04-09', '04-10', '04-11',
       '04-12', '04-13', '04-14', '04-15', '04-16', '04-17', '04-18',
       '04-19', '04-20', '04-21', '04-22', '04-23\n04-24', '05-01',
       '05-02', '05-03',

In [22]:
print("Ratings exist for ", len(ratings['S-E'].unique()), " episodes")
print("Viewership is for ", len(viewership['Episode'].unique()), " episodes")
print("Dialogue set has ", len(df['S-E'].unique()), " episodes")

Ratings exist for  236  episodes
Viewership is for  228  episodes
Dialogue set has  227  episodes


The inconsistency in episode count across datasets is due to the fact that

- Dialogue has merged scripts for 2 part episodes
- Viewership has merged 2 parters into same viewership, and added special episode
- Ratings has individual ratings for all episodes except special

Hence, we remove speical from viewership and take average rating of 2 part episodes in final

### Fix Viewership DataFrame

In [23]:
# First, fix viewership dataframe
viewership.drop(viewership[viewership.Episode == 'Special'].index, inplace=True)
viewership['S-E'] = viewership.Episode.str[:5]

print(viewership.shape)
viewership['S-E'].unique()

(227, 11)


array(['01-01', '01-02', '01-03', '01-04', '01-05', '01-06', '01-07',
       '01-08', '01-09', '01-10', '01-11', '01-12', '01-13', '01-14',
       '01-15', '01-16', '01-17', '01-18', '01-19', '01-20', '01-21',
       '01-22', '01-23', '01-24', '02-01', '02-02', '02-03', '02-04',
       '02-05', '02-06', '02-07', '02-08', '02-09', '02-10', '02-11',
       '02-12', '02-14', '02-15', '02-16', '02-17', '02-18', '02-19',
       '02-20', '02-21', '02-22', '02-23', '02-24', '03-01', '03-02',
       '03-03', '03-04', '03-05', '03-06', '03-07', '03-08', '03-09',
       '03-10', '03-11', '03-12', '03-13', '03-14', '03-15', '03-16',
       '03-17', '03-18', '03-19', '03-20', '03-21', '03-22', '03-23',
       '03-24', '03-25', '04-01', '04-02', '04-03', '04-04', '04-05',
       '04-06', '04-07', '04-08', '04-09', '04-10', '04-11', '04-12',
       '04-13', '04-14', '04-15', '04-16', '04-17', '04-18', '04-19',
       '04-20', '04-21', '04-22', '04-23', '05-01', '05-02', '05-03',
       '05-04', '05-

In [24]:
# Empty list -> no missing elements
s = set(df['S-E'].unique())
final_list = [x for x in viewership['S-E'].unique() if x not in s]
final_list

[]

In [25]:
# Next, extract viewers

viewership['viewers'] = viewership['U.S. viewers'].str.split(" ", expand=True)[0].astype(float)
viewership

,Date,Episode,Title,Directed by,Written by,Duration,Summary,Rating/Share,U.S. viewers,Prod.\ncode,S-E,viewers
0,09/22/94,01-01,The Pilot,James Burrows,David Crane & Marta Kauffman,22.0,"Monica and the gang introduce Rachel to the ""r...",14.7/23,21.5 million,456650,01-01,21.50
1,09/29/94,01-02,The One with the Sonogram at the End,James Burrows,David Crane & Marta Kauffman,22.0,Ross finds out his ex-wife is pregnant. Rachel...,14.0/22,20.2 million,456652,01-02,20.20
2,10/06/94,01-03,The One with the Thumb,James Burrows,Jeffrey Astrof & Mike Sikowitz,22.0,Monica becomes irritated when everyone likes h...,13.6/22,19.5 million,456651,01-03,19.50
3,10/13/94,01-04,The One with George Stephanopoulos,James Burrows,Alexa Junge,22.0,Joey and Chandler take Ross to a hockey game t...,13.7/22,19.7 million,456654,01-04,19.70
4,10/20/94,01-05,The One with the East German Laundry Detergent,Pamela Fryman,Jeff Greenstein & Jeff Strauss,22.0,"Eager to spend time with Rachel, Ross pretends...",12.9/20,18.6 million,456653,01-05,18.60
...,...,...,...,...,...,...,...,...,...,...,...,...
223,02/19/04,10-13,The One Where Joey Speaks French,Gary Halvorson,Sherry Bilsing-Graham & Ellen Plummer,22.0,Phoebe tries to teach Joey to speak French for...,15.2/24,24.27 million,176261,10-13,24.27
224,02/26/04,10-14,The One with Princess Consuela,Gary Halvorson,Story by : Robert Carlock\nTeleplay by : Tracy...,22.0,When Phoebe goes to get her name changed she r...,14.6/22,22.83 million,176263,10-14,22.83
225,04/22/04,10-15,The One Where Estelle Dies,Gary Halvorson,Story by : Mark Kunerth\nTeleplay by : David C...,22.0,Ross tries to get Rachel to go back to Ralph L...,14.4/24,22.64 million,176264,10-15,22.64
226,04/29/04,10-16,The One with Rachel's Going Away Party,Gary Halvorson,Andrew Reich & Ted Cohen,22.0,"The gang throws Rachel a goodbye party, during...",15.5/25,24.51 million,176265,10-16,24.51


### Fix Ratings DataFrame

In [26]:
# Elements in ratings that aren't in scripts dataframe
s = set(df['S-E'].unique())
missing_list = [x for x in ratings['S-E'].unique() if x not in s]
missing_list

['02-13',
 '04-24',
 '05-24',
 '06-16',
 '06-25',
 '07-24',
 '08-24',
 '09-24',
 '10-18']

In [27]:
for i in missing_list:
    idx = ratings[ratings['S-E'] == i].index[0]
    s, e = i.split('-')
    row_above = s + '-' + str(int(e)-1)
    
    row_stars = float(ratings[ratings['S-E'] == i].Stars)
    row_votes = int(ratings[ratings['S-E'] == i].Votes)
    above_stars = float(ratings[ratings['S-E'] == row_above].Stars)
    above_votes = int(ratings[ratings['S-E'] == row_above].Votes)
    
    ratings.at[idx - 1, 'Stars'] = ((row_stars * row_votes) + (above_stars * above_votes)) / (row_votes + above_votes)
    ratings.at[idx - 1, 'Votes'] = (row_votes + above_votes) // 2
    
    ratings.drop(index=idx, inplace=True)

In [28]:
ratings

,Year_of_prod,Season,Episode,Episode_Title,Duration,Summary,Director,Stars,Votes,S-E
0,1994,01,01,The One Where Monica Gets a Roommate: The Pilot,22,"Monica and the gang introduce Rachel to the ""r...",James Burrows,8.300000,7440,01-01
1,1994,01,02,The One with the Sonogram at the End,22,Ross finds out his ex-wife is pregnant. Rachel...,James Burrows,8.100000,4888,01-02
2,1994,01,03,The One with the Thumb,22,Monica becomes irritated when everyone likes h...,James Burrows,8.200000,4605,01-03
3,1994,01,04,The One with George Stephanopoulos,22,Joey and Chandler take Ross to a hockey game t...,James Burrows,8.100000,4468,01-04
4,1994,01,05,The One with the East German Laundry Detergent,22,"Eager to spend time with Rachel, Ross pretends...",Pamela Fryman,8.500000,4438,01-05
...,...,...,...,...,...,...,...,...,...,...
230,2004,10,13,The One Where Joey Speaks French,22,Phoebe tries to teach Joey to speak French for...,Gary Halvorson,8.500000,3044,10-13
231,2004,10,14,The One with Princess Consuela,22,When Phoebe goes to get her name changed she r...,Gary Halvorson,8.600000,2989,10-14
232,2004,10,15,The One Where Estelle Dies,22,Ross tries to get Rachel to go back to Ralph L...,Gary Halvorson,8.500000,2771,10-15
233,2004,10,16,The One with Rachel's Going Away Party,22,"The gang throws Rachel a goodbye party, during...",Gary Halvorson,8.900000,3141,10-16


In [29]:
# Empty list -> no missing elements
s = set(df['S-E'].unique())
final_list = [x for x in ratings['S-E'].unique() if x not in s]
final_list

[]

In [30]:
# Check if both viewership and ratings are sorted the same way
viewership['S-E'].values == ratings['S-E'].values

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,

#### Create final metadata dataframe

In [31]:
metadata = pd.DataFrame()
metadata['S-E'] = ratings['S-E'].values
metadata['Viewers'] = viewership['viewers'].values
metadata['Stars'] = ratings['Stars'].values

metadata

,S-E,Viewers,Stars
0,01-01,21.50,8.300000
1,01-02,20.20,8.100000
2,01-03,19.50,8.200000
3,01-04,19.70,8.100000
4,01-05,18.60,8.500000
...,...,...,...
222,10-13,24.27,8.500000
223,10-14,22.83,8.600000
224,10-15,22.64,8.500000
225,10-16,24.51,8.900000


# 3. Saving the CSV files

In [43]:
df

,Season,Episode,Title,Speaker,Dialog,S-E
1,04,10,The Girl From Poughkeepsie,Ross,hey,04-10
3,04,10,The Girl From Poughkeepsie,Ross,im sorry im late did i miss anything,04-10
4,04,10,The Girl From Poughkeepsie,Phoebe,joey stuffing 15 oreos in his mouth,04-10
5,04,10,The Girl From Poughkeepsie,Ross,15 your personal best,04-10
6,04,10,The Girl From Poughkeepsie,Phoebe,where were you,04-10
...,...,...,...,...,...,...
61598,03,25,At The Beach,Ross,hi,03-25
61599,03,25,At The Beach,Chandler,hi there,03-25
61600,03,25,At The Beach,Monica,thats that weird voice again,03-25
61601,03,25,At The Beach,Chandler,okay okay let me try it again youre gonna wann...,03-25


In [41]:
metadata

,S-E,Viewers,Stars
0,01-01,21.50,8.300000
1,01-02,20.20,8.100000
2,01-03,19.50,8.200000
3,01-04,19.70,8.100000
4,01-05,18.60,8.500000
...,...,...,...
222,10-13,24.27,8.500000
223,10-14,22.83,8.600000
224,10-15,22.64,8.500000
225,10-16,24.51,8.900000


In [50]:
df.to_csv('data/processed_scripts.csv', index=False)
metadata.to_csv('data/metadata.csv', index=False)